In [ ]:
! pip install sqlalchemy pyodbc

In [242]:
import sqlalchemy
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest,RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [220]:
server = r"SAKS\SQLEXPRESS"
database = "HEALTHCARE_"
username = "Saks"
password = "Sql@2025"

In [221]:
# Creating SQLAlchemy engine using PyODBC
engine = sqlalchemy.create_engine(f"mssql+pyodbc://{server}/{database}?driver=SQL+Server&trusted_connection=yes") 

In [ ]:
# Fetching all table names
query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'"
df_tables = pd.read_sql(query, engine)

print(df_tables)

# Fetching data from a specific table
table_name = "Medicare_Charge_Inpatient"  
Medicare_Charge_Inpatient = pd.read_sql(f"SELECT * FROM {table_name}", engine)

table_name = "Medicare_Charge_Outpatient"  
Medicare_Charge_Outpatient = pd.read_sql(f"SELECT * FROM {table_name}", engine)

table_name = "Medicare_Provider_Charge_Inpatient"  
Medicare_Provider_Charge_Inpatient = pd.read_sql(f"SELECT * FROM {table_name}", engine)

table_name = "Medicare_Provider_Charge_Outpatient"  
Medicare_Provider_Charge_Outpatient = pd.read_sql(f"SELECT * FROM {table_name}", engine)

table_name = "Patient_history_samp"  
Patient_history_samp = pd.read_sql(f"SELECT * FROM {table_name}", engine)

table_name = "Review_patient_history_samp"  
Review_patient_history_samp = pd.read_sql(f"SELECT * FROM {table_name}", engine)

table_name = "Review_transaction_coo"  
Medicare_Charge_Inpatient = pd.read_sql(f"SELECT * FROM {table_name}", engine)

table_name = "Transaction_coo"  
Transaction_coo = pd.read_sql(f"SELECT * FROM {table_name}", engine)

In [ ]:
Medicare_Provider_Charge_Inpatient.head(1)

In [224]:
Medicare_Provider_Charge_Inpatient.rename(columns={"Average_Total_Payments":"Average_Total_Payments_In","Provider_Name" : "Provider_Name_In","Provider_Street_Address":"Provider_Street_Address_In","Provider_City":"Provider_City_In","Provider_State":"Provider_State_In","Provider_Zip_Code":"Provider_Zip_Code_In","Hospital_Referral_Region_HRR_Description":"Hospital_Referral_Region_HRR_Description_In"},inplace=True)

Medicare_Provider_Charge_Outpatient.rename(columns={"Average_Total_Payments":"Average_Total_Payments_Out","Provider_Name" : "Provider_Name_Out","Provider_Street_Address":"Provider_Street_Address_Out","Provider_City":"Provider_City_Out","Provider_State":"Provider_State_Out","Provider_Zip_Code":"Provider_Zip_Code_Out","Hospital_Referral_Region_HRR_Description":"Hospital_Referral_Region_HRR_Description_Out"},inplace=True)

In [225]:
# Merge datasets on Provider_Id
merged_data = Medicare_Provider_Charge_Inpatient.merge(Medicare_Provider_Charge_Outpatient, on="Provider_Id")

In [ ]:
pd.set_option('display.max_columns', None)
merged_data.head(3)

Identify providers that overcharge for certain procedures or regions where procedures are too expensive.

Highest Cost Variation

In [ ]:
cost_variation = Medicare_Provider_Charge_Inpatient.groupby("DRG_Definition")["Average_Total_Payments_In"].std()
most_variable_procedure = cost_variation.idxmax()
print("Most variable procedure: ", most_variable_procedure)

Highest Cost claims by Provider

In [ ]:
# Compute Z-score within each procedure
Medicare_Provider_Charge_Inpatient["Payment_ZScore"] = Medicare_Provider_Charge_Inpatient.groupby("DRG_Definition")["Average_Total_Payments_In"].transform(zscore)

# Identify overcharging providers (Z-Score > 3)
overcharging_providers = Medicare_Provider_Charge_Inpatient[Medicare_Provider_Charge_Inpatient["Payment_ZScore"] > 3][["Provider_Id", "DRG_Definition", "Average_Total_Payments_In", "Payment_ZScore"]]

top_providers = overcharging_providers.sort_values(by="Payment_ZScore", ascending=False).head(10)

# Bar Chart: Top Overcharging Providers

Medicare_Provider_Charge_Inpatient["DRG_Definition"] = pd.Categorical(Medicare_Provider_Charge_Inpatient["DRG_Definition"], ordered=True)

top__charging_providers = top_providers.sort_values(by="Average_Total_Payments_In",ascending=False)

top__charging_providers

# these are the top 10 procedure with highest cost variaiton for inpatient

In [197]:
# Note -- z-score will find which provider overcharge for specific procedure compared to others
# std -  will find the cost variation amoung different provider for each procedure and find which procedure ave highest cost variation amoung providers (overalanga a procedure la highest cost variation ithavo athuna std kamchathu)example -
# DRG_Definition	Provider	Average_Total_Payments_In
# Heart Surgery	A	$10,000
# Heart Surgery	B	$12,000
# Heart Surgery	C	$25,000
# Knee Surgery	D	$7,000
# Knee Surgery	E	$7,200
# Knee Surgery	F	$7,500
# Heart Surgery has a big price difference ($10,000 to $25,000).
# Knee Surgery prices are close ($7,000 to $7,500).
# So, std for Heart Surgery > Knee Surgery → More variation in charges.

Highest Cost Claims by Region

In [ ]:
highest_cost_regions = Medicare_Provider_Charge_Inpatient.groupby("Provider_State_In")[["Average_Covered_Charges","Average_Total_Payments_In"]].mean().sort_values(by ="Average_Covered_Charges" ,ascending=False).head(10)

print("Top 10 Highest-Cost Regions:\n")
highest_cost_regions


In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
highest_cost_regions.plot(kind="bar", figsize=(14, 6), width=0.8, colormap="coolwarm", edgecolor="black")

plt.title("Top 10 Highest-Cost Regions: Covered Charges vs. Total Payments", fontsize=14)
plt.xlabel("Hospital Referral Region (HRR)", fontsize=12)
plt.ylabel("Mean Amount ($)", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.legend(["Average Covered Charges", "Average Total Payments"], fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

In [ ]:
highest_cost_regions_hrr = (Medicare_Provider_Charge_Inpatient.groupby("Hospital_Referral_Region_HRR_Description_In")[["Average_Covered_Charges","Average_Total_Payments_In"]].mean().sort_values(by='Average_Covered_Charges',ascending=False)).head(10)

print("Top 10 Highest-Cost HRR Regions:\n")

highest_cost_regions_hrr


In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
highest_cost_regions_hrr.plot(kind="bar", figsize=(14, 6), width=0.8, colormap="coolwarm", edgecolor="black")

plt.title("Top 10 Highest-Cost HRR Regions: Covered Charges vs. Total Payments", fontsize=14)
plt.xlabel("Hospital Referral Region (HRR)", fontsize=12)
plt.ylabel("Mean Amount ($)", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.legend(["Average Covered Charges", "Average Total Payments"], fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()


Highest Number of Procedures and Largest Differences between Claims and Reimbursements


In [ ]:
# Highest Number of Procedures

procedure_count_in = Medicare_Provider_Charge_Inpatient.groupby("DRG_Definition")['Total_Discharges'].sum().reset_index()

procedure_count_in.sort_values(by="Total_Discharges",ascending=False).head(10)

In [ ]:
# Largest Difference between claims and reimbursement 

Medicare_Provider_Charge_Inpatient["Diff_btw_claim_reimburs"] = Medicare_Provider_Charge_Inpatient['Average_Covered_Charges']-Medicare_Provider_Charge_Inpatient['Average_Medicare_Payments']

claim_diff_by_provider = Medicare_Provider_Charge_Inpatient.groupby("Provider_Id")["Diff_btw_claim_reimburs"].mean().reset_index()

claim_diff_by_provider.sort_values(by="Diff_btw_claim_reimburs", ascending=False).head(10)

Finding 3 Providers least like others

In [204]:
# Select numerical features
features = ["Outpatient_Services", "Average_Estimated_Submitted_Charges", "Average_Total_Payments_Out","Average_Total_Payments_In","Total_Discharges", "Average_Covered_Charges", "Average_Medicare_Payments"]

In [ ]:
data = merged_data[["Provider_Id"] + features]
data.head(5)

In [ ]:
# check for null values
data.isna().sum()

In [207]:
# Normalize data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[features])

In [208]:
# Apply Isolation Forest for anomaly detection
iso_forest = IsolationForest(contamination=0.01, random_state=42)
anomaly_scores = iso_forest.fit_predict(data_scaled)

In [ ]:
# Identify least similar providers (anomalies)
data = data.copy()
data.loc[:,"Anomaly_Score"] = anomaly_scores
least_similar_providers = data[data["Anomaly_Score"] == -1].sort_values(by="Anomaly_Score").head(3)
print("Least Similar Providers:")
print(least_similar_providers[["Provider_Id"]])

 Finding the 3 regions least like others


In [210]:
# Define Features for Each Category
inpatient_features = ["Total_Discharges", "Average_Covered_Charges", "Average_Total_Payments_In", "Average_Medicare_Payments"]
outpatient_features = ["Outpatient_Services", "Average_Estimated_Submitted_Charges", "Average_Total_Payments_Out"]


In [211]:
# Aggregate Data by Region
df_inpatient_region = Medicare_Provider_Charge_Inpatient.groupby("Hospital_Referral_Region_HRR_Description_In")[inpatient_features].mean().reset_index()
df_outpatient_region = Medicare_Provider_Charge_Outpatient.groupby("Hospital_Referral_Region_HRR_Description_Out")[outpatient_features].mean().reset_index()

In [212]:
# Normalize Data
scaler = StandardScaler()
df_inpatient_scaled = scaler.fit_transform(df_inpatient_region[inpatient_features]) #Normalizing the numerical values
df_outpatient_scaled = scaler.fit_transform(df_outpatient_region[outpatient_features]) 

In [213]:
# Apply Isolation Forest for Inpatient
iso_forest_inpatient = IsolationForest(contamination=0.01, random_state=42)
df_inpatient_region["Anomaly_Score"] = iso_forest_inpatient.fit_predict(df_inpatient_scaled)

# Apply Isolation Forest for Outpatient
iso_forest_outpatient = IsolationForest(contamination=0.01, random_state=42)
df_outpatient_region["Anomaly_Score"] = iso_forest_outpatient.fit_predict(df_outpatient_scaled)

In [ ]:
# Get the Three Least Similar Regions for Inpatient
least_similar_inpatient_regions = df_inpatient_region[df_inpatient_region["Anomaly_Score"] == -1].nsmallest(3, "Anomaly_Score")

# Get the Three Least Similar Regions for Outpatient
least_similar_outpatient_regions = df_outpatient_region[df_outpatient_region["Anomaly_Score"] == -1].nsmallest(3, "Anomaly_Score")

print("Three Regions Least Like Others (Inpatient):")
print(least_similar_inpatient_regions[["Hospital_Referral_Region_HRR_Description_In"]])
print("------------------------------------------------------")
print("Three Regions Least Like Others (Outpatient):")
print(least_similar_outpatient_regions[["Hospital_Referral_Region_HRR_Description_Out"]])

Identify 10,000 Medicare patients who are involved in anomalous activities. 

### **🔍 Predictive Analytics Problem Statements (Healthcare & Medicare Fraud)**
Since you are working with **Medicare payment data**, you can explore several predictive analytics problems. Here are some problem statements:

---

### **1️⃣ Fraud Detection in Medicare Claims**  
**Problem Statement:**  
Predict whether a **Medicare claim** is fraudulent based on billing patterns, claim amounts, and provider history.  
**Model:** Classification (Logistic Regression, Random Forest, XGBoost, Isolation Forest for anomalies)  
**Dataset Features:**  
- Provider ID  
- Procedure Type  
- Total Charges vs. Medicare Payments  
- Frequency of Claims by Provider  
- Outlier Detection in Billing  

---

### **2️⃣ Predicting Overpriced Procedures**  
**Problem Statement:**  
Predict if a hospital **overcharges for a procedure** compared to industry standards.  
**Model:** Regression (Linear Regression, XGBoost, Random Forest)  
**Dataset Features:**  
- **Average Covered Charges** (Cost to patient)  
- **Average Medicare Payments** (Actual reimbursement)  
- **Total Discharges** (Hospital workload)  
- **Geographical Impact** (Region-wise variations)  

---

### **3️⃣ Identifying Patients at Risk of Readmission**  
**Problem Statement:**  
Predict whether a **Medicare patient** will be readmitted within 30 days after discharge.  
**Model:** Classification (Logistic Regression, Random Forest, Neural Networks)  
**Dataset Features:**  
- Diagnosis & Procedure Codes  
- Total Charges & Payments  
- Previous Readmission History  
- Length of Stay  
- Patient Age & Health Conditions  

---

### **4️⃣ Predicting Patient’s Length of Stay**  
**Problem Statement:**  
Estimate the **number of days** a patient will stay in a hospital based on their condition.  
**Model:** Regression (Random Forest, Decision Trees, XGBoost)  
**Dataset Features:**  
- Admission Type (Emergency, Elective)  
- Diagnosis & Procedures  
- Past Medical History  
- Insurance Coverage  

---

### **5️⃣ Predicting Mortality Rate Based on Hospital Data**  
**Problem Statement:**  
Predict the probability of **patient mortality** based on hospital stay, treatment, and payments.  
**Model:** Classification (Random Forest, Neural Networks)  
**Dataset Features:**  
- Age & Comorbidities  
- Procedure Type  
- Length of Hospital Stay  
- ICU Admission  

---

### **📌 Would You Like to Build One?**  
I can help you:  
✅ **Choose the best problem** for your dataset.  
✅ **Train a model** using ML algorithms.  
✅ **Deploy the model** using **Streamlit** for real-time predictions.  

Which problem do you want to solve? 🚀

# Fraud Detection in Medicare Claims
(Identify whether a Medicare claim is fraudulent based on provider billing patterns.)

In [243]:
# Feature Engineering
Medicare_Provider_Charge_Inpatient["Overcharging_Ratio"] = Medicare_Provider_Charge_Inpatient["Average_Covered_Charges"] / Medicare_Provider_Charge_Inpatient["Average_Total_Payments_In"]

Medicare_Provider_Charge_Inpatient["Claim_Difference"] = Medicare_Provider_Charge_Inpatient["Average_Covered_Charges"] - Medicare_Provider_Charge_Inpatient["Average_Total_Payments_In"]

In [246]:
# Detect Anomalies using Isolation Forest
iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
Medicare_Provider_Charge_Inpatient["Anomaly_Score"] = iso_forest.fit_predict(Medicare_Provider_Charge_Inpatient[["Overcharging_Ratio", "Claim_Difference", "Payment_ZScore"]])

In [247]:
# Map -1 as Anomalous (Potential Fraud), 1 as Normal
Medicare_Provider_Charge_Inpatient["Fraud_Label"] = Medicare_Provider_Charge_Inpatient["Anomaly_Score"].map({-1: 1, 1: 0})

In [248]:
# Train Fraud Detection Model (If labels exist)
if "Fraud_Label" in Medicare_Provider_Charge_Inpatient.columns:
    X = Medicare_Provider_Charge_Inpatient[["Overcharging_Ratio", "Claim_Difference", "Payment_ZScore"]]
    y = Medicare_Provider_Charge_Inpatient["Fraud_Label"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)

    print("Model Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

Model Accuracy: 0.9973630147487198
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30979
           1       0.98      0.97      0.97      1634

    accuracy                           1.00     32613
   macro avg       0.99      0.98      0.99     32613
weighted avg       1.00      1.00      1.00     32613



In [254]:
# Creating a pickle file
import pickle
picked_out = open("Classifier.pkl","wb")
pickle.dump(rf_model,picked_out)
picked_out.close()